# Dataset Management and Labeling and Feature Extraction

 La base de datos fue organizada en directorios etiquetados para facilitar su manejo y acceso, consolidando los datos con ruido y sin ruido.
 
 Posteriormente, se extrajeron las características de cada archivo de audio, calculando el espectrograma de Mel para capturar la esencia frecuencial de las señales. Se estandarizaron las muestras mediante relleno de ceros para unificar las dimensiones, resultando en un conjunto de 18,908 características con sus respectivas etiquetas.

In [1]:
import librosa
import numpy as np
import os

def pad_audio(audio, max_length):
    return np.pad(audio, (0, max(0, max_length - len(audio))), mode='constant')

def extract_mel_spectrogram(file_path, max_length=16000, n_mels=64, n_fft=2048, hop_length=512):
    try:
        audio, sr = librosa.load(file_path, sr=None)
        audio = pad_audio(audio, max_length)
        S = librosa.feature.melspectrogram(y=audio, sr=sr, n_mels=n_mels, n_fft=n_fft, hop_length=hop_length)
        S_DB = librosa.power_to_db(S, ref=np.max)
        return S_DB.T  
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return None

def load_data(directory):
    features = []
    labels = []

    for subdir, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.wav'):
                file_path = os.path.join(subdir, file)
                mel_spectrogram = extract_mel_spectrogram(file_path)
                if mel_spectrogram is not None:
                    features.append(mel_spectrogram)
                    labels.append(os.path.basename(subdir))  # The directory name is the label

    return np.array(features), np.array(labels)

# Paths to the datasets
clean_directory = '/Users/marcgrayson/Procesamiento de Datos/direction_commands.extracted.tar'
noisy_directory = '/Users/marcgrayson/Procesamiento de Datos/direction_commands_with_noise'

# Loading the datasets
clean_features, clean_labels = load_data(clean_directory)
noisy_features, noisy_labels = load_data(noisy_directory)

# Combine features and labels from both datasets
combined_features = np.concatenate((clean_features, noisy_features), axis=0)
combined_labels = np.concatenate((clean_labels, noisy_labels), axis=0)

# Checking the number of features and labels to ensure they match
print(f"Total number of features: {len(combined_features)}")
print(f"Total number of labels: {len(combined_labels)}")


Total number of features: 18908
Total number of labels: 18908


# Feature Selection  
Se planea continuar con la selección de características utilizando SelectKBest y la prueba ANOVA F-value para identificar y retener las 100 características más significativas.

In [9]:
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(combined_labels)

# SelectKBest con la prueba ANOVA F-value
# Seleccionar mejores caracteristicas k

selector = SelectKBest(f_classif, k=100)
#X_selected = selector.fit_transform(combined_features, encoded_labels)

#print(f"Total number of features before selection: {combined_features.shape[1]}")
#print(f"Total number of features after selection: {X_selected.shape[1]}")


# Train Model

Con las características seleccionadas, se procederá al entrenamiento de un modelo de red neuronal convolucional. Este modelo se compone de varias capas convolucionales y de agrupamiento, seguidas de capas densas. 

La arquitectura del modelo ha sido configurada para la clasificación múltiple, con una capa de salida que utiliza la función de activación 'softmax' para predecir las probabilidades de cada comando de voz.

El modelo se compila con una función de pérdida de entropía cruzada categórica y el optimizador Adam, una elección adecuada para tareas de clasificación con múltiples clases. 

In [ ]:
# Crear el modelo de red convolucional 1D
model = Sequential()
model.add(Conv1D(16, 3, activation='relu', input_shape=(X_train_selected.shape[1], 1)))
model.add(MaxPooling1D(2))
model.add(Dropout(0.25))
model.add(Conv1D(32, 3, activation='relu'))
model.add(MaxPooling1D(2))
model.add(Dropout(0.25))
model.add(Conv1D(32, 3, activation='relu'))
model.add(MaxPooling1D(2))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(len(class_labels), activation='softmax'))  # Cambio para clasificación múltiple

# Compilar el modelo
model.compile(loss='categorical_crossentropy',  # Cambio para clasificación múltiple
              optimizer='adam',
              metrics=['accuracy'])

# Entrenar el modelo
history = model.fit(X_train_selected, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_data=(X_test_selected, np_utils.to_categorical(y_test)))

# Evaluate Model

La evaluación del modelo, prevista como el siguiente paso, implicará el uso de un conjunto de datos de prueba para medir su precisión y capacidad de clasificación. Se utilizarán métricas como precisión, sensibilidad y puntuación F1, y se creará una matriz de confusión para detectar y analizar los errores de clasificación. 

In [ ]:
# Evaluación del modelo
y_pred = model.predict(X_test_selected)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(np_utils.to_categorical(y_test), axis=1)

print(classification_report(y_true, y_pred_classes, target_names=class_labels))
conf_matrix = confusion_matrix(y_true, y_pred_classes)
print(conf_matrix)
